In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import random
import time

import argparse

import rdkit
from rdkit import Chem
from rdkit.Chem import ChemicalFeatures
from rdkit import RDConfig

import shutil
import os
from os import listdir
from os.path import isfile, join
import xml.etree.ElementTree as ET

# Torch
import torch
import torch.optim as optim
import torch.nn as nn
from torch.autograd import Variable
from torch.autograd.variable import Variable
import torch.nn.functional as F

#import multiprocessing
#from joblib import Parallel, delayed

from __future__ import print_function

from sklearn.metrics import classification_report

In [2]:
print(torch.__version__)
!nvidia-smi

1.12.1+cu102
Thu Nov 17 04:50:53 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 520.61.05    Driver Version: 520.61.05    CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:1E.0 Off |                    0 |
| N/A   62C    P8    17W /  70W |      2MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                           

In [3]:
torch.cuda.is_available()

True

In [4]:
!gpustat

ip-10-0-101-229  Thu Nov 17 04:50:54 2022  520.61.05
[0] Tesla T4 | 62°C,   0 % |   404 / 15360 MB |


In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [5]:
import pickle
from skmultilearn.dataset import load_from_arff, load_dataset_dump
import copy
import datetime
import jsonpickle

In [6]:
import numpy as np
from scipy.sparse import lil_matrix
from sklearn.model_selection import KFold, StratifiedKFold
import pandas as pd
import copy
from itertools import chain

In [7]:
from builtins import str
from builtins import range
from builtins import object
import arff
import bz2
import numpy as np
import os
import csv
import sys
import shutil
from os import environ
from os.path import dirname
from os.path import join
from os.path import exists
from os.path import expanduser
from os.path import isdir
from os.path import splitext
from os import listdir
from os import makedirs
from scipy import sparse


In [8]:
# Load data

In [9]:
dat_label = pd.read_csv('dataset.csv',index_col=0)
#label = label.iloc[:12,:]
dat_smiles = pd.read_csv('smiles.csv', index_col=0)
s = dat_smiles.to_dict()
dict_smile = s['SMILES']

In [10]:
dat_labels = dat_label.T

del 11980947,5287407,24238,44475014,62074,11686063,133082064,16211610,173849,CID23308253,24753271

In [11]:
#dat_label = pd.concat([dat_label.loc['fruity'],1-dat_label.loc['fruity']],axis = 1)
dat_label = pd.concat([dat_label.loc['odorless'],1-dat_label.loc['odorless']],axis = 1)
dat_label = dat_label.T

In [12]:
# Parser check
def restricted_float(x, inter):
    x = float(x)
    if x < inter[0] or x > inter[1]:
        raise argparse.ArgumentTypeError("%r not in range [1e-5, 1e-4]"%(x,))
    return x

# Argument parser
import sys
sys.argv=['']
del sys

parser = argparse.ArgumentParser(description='Neural message passing')

parser.add_argument('--dataset', default='qm9', help='QM9')
parser.add_argument('--datasetPath', default='/home/taobai/Documents/Model/MPNN/mpnn-data/qm9/dsgdb9nsd/', help='dataset path')
parser.add_argument('--logPath', default='./log/qm9/mpnn/', help='log path')
parser.add_argument('--plotLr', default=False, help='allow plotting the data')
parser.add_argument('--plotPath', default='./plot/qm9/mpnn/', help='plot path')
parser.add_argument('--resume', default='./checkpoint/qm9/mpnn/',
                    help='path to latest checkpoint')

# Optimization Options
parser.add_argument('--batch-size', type=int, default=10, metavar='N',
                    help='Input batch size for training (default: 20)')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='Enables CUDA training')
parser.add_argument('--epochs', type=int, default=200, metavar='N',
                    help='Number of epochs to train (default: 100)')
parser.add_argument('--lr', type=lambda x: restricted_float(x, [1e-5, 1e-2]), default=1e-3, metavar='LR',
                    help='Initial learning rate [1e-5, 5e-4] (default: 1e-4)')
parser.add_argument('--lr-decay', type=lambda x: restricted_float(x, [.01, 1]), default=0.6, metavar='LR-DECAY',
                    help='Learning rate decay factor [.01, 1] (default: 0.6)')
parser.add_argument('--schedule', type=list, default=[0.1, 0.9], metavar='S',
                    help='Percentage of epochs to start the learning rate decay [0, 1] (default: [0.1, 0.9])')
parser.add_argument('--momentum', type=float, default=0.9, metavar='M',
                    help='SGD momentum (default: 0.9)')
# i/o
parser.add_argument('--log-interval', type=int, default=20, metavar='N',
                    help='How many batches to wait before logging training status')
# Accelerating
parser.add_argument('--prefetch', type=int, default=4, help='Pre-fetching threads.')

# Model modification
parser.add_argument('--model', type=str,help='MPNN model name [MPNN, MPNNv2, MPNNv3]',
                        default='MPNN')

_StoreAction(option_strings=['--model'], dest='model', nargs=None, const=None, default='MPNN', type=<class 'str'>, choices=None, help='MPNN model name [MPNN, MPNNv2, MPNNv3]', metavar=None)

In [13]:
args = parser.parse_args()
print(args)

Namespace(dataset='qm9', datasetPath='/home/taobai/Documents/Model/MPNN/mpnn-data/qm9/dsgdb9nsd/', logPath='./log/qm9/mpnn/', plotLr=False, plotPath='./plot/qm9/mpnn/', resume='./checkpoint/qm9/mpnn/', batch_size=10, no_cuda=False, epochs=200, lr=0.001, lr_decay=0.6, schedule=[0.1, 0.9], momentum=0.9, log_interval=20, prefetch=4, model='MPNN')


In [14]:
args.cuda = not args.no_cuda and torch.cuda.is_available()

In [15]:
best_er1 = 0

In [16]:
def xyz_graph_reader(CID):
    smiles = dict_smile[CID]
    m = Chem.MolFromSmiles(smiles)
    m = Chem.AddHs(m)
   
    g = nx.Graph()
    # Create nodes
    for i in range(0, m.GetNumAtoms()):
        atom_i = m.GetAtomWithIdx(i)

        g.add_node(i, a_type=atom_i.GetSymbol(), a_num=atom_i.GetAtomicNum(), acceptor=0, donor=0,
                   aromatic=atom_i.GetIsAromatic(), hybridization=atom_i.GetHybridization(),
                   num_h=atom_i.GetTotalNumHs())


    # Read Edges
    for i in range(0, m.GetNumAtoms()):
        for j in range(0, m.GetNumAtoms()):
            e_ij = m.GetBondBetweenAtoms(i, j)
            if e_ij is not None:
                g.add_edge(i, j, b_type=e_ij.GetBondType())
            else:
                # Unbonded
                g.add_edge(i, j, b_type=None)

    l = list(dat_label[CID].values)           
                
    return g , l

In [17]:
from sklearn.model_selection import train_test_split

train_ids, stratified_ids = train_test_split(dat_label.T.index, test_size=0.2,)
valid_ids, test_ids = train_test_split(stratified_ids, test_size=0.5,)
#train_ids =  np.random.choice(train_ids, size=1000, replace=False)
#valid_ids =  np.random.choice(valid_ids, size=100, replace=False)
#test_ids = np.random.choice(valid_ids, size=100, replace=False)

In [18]:
def qm9_edges(g):
    remove_edges = []
    e={}    
    for n1, n2, d in g.edges(data=True):
        e_t = []
        # Raw distance function
        if d['b_type'] is None:
            remove_edges += [(n1, n2)]
        else:
            #e_t.append(d['distance'])
            e_t += [int(d['b_type'] == x) for x in [rdkit.Chem.rdchem.BondType.SINGLE, rdkit.Chem.rdchem.BondType.DOUBLE,
                                                    rdkit.Chem.rdchem.BondType.TRIPLE, rdkit.Chem.rdchem.BondType.AROMATIC]]
        if e_t:
            e[(n1, n2)] = e_t
    for edg in remove_edges:
        g.remove_edge(*edg)    
    
    return nx.to_numpy_matrix(g), e
    

In [19]:
def qm9_nodes(g, hydrogen=False):
    h = []
    for n, d in g.nodes(data=True): 
        h_t = []
        # Atom type (One-hot H, C, N, O F)
        h_t += [int(d['a_type'] == x) for x in ['H', 'C', 'N', 'O', 'F']]
        # Atomic number
        h_t.append(d['a_num'])
        # Acceptor
        h_t.append(d['acceptor'])
        # Donor
        h_t.append(d['donor'])
        # Aromatic
        h_t.append(int(d['aromatic']))
        # If number hydrogen is used as a
        if hydrogen:
            h_t.append(d['num_h'])
        h.append(h_t)
    return h

In [20]:
class Qm9():
#class Qm9(data.Dataset):
    # Constructor
    def __init__(self, idx, vertex_transform=qm9_nodes, edge_transform=qm9_edges,
                 target_transform=None, e_representation='raw_distance'):
        self.idx = idx
        self.vertex_transform = vertex_transform
        self.edge_transform = edge_transform
        self.target_transform = target_transform
        #化学分子距离度量方式，有三种，这里用的是'raw_distance'
        self.e_representation = e_representation

    def __getitem__(self,index):
        #读图，返回networkx类型的图，标签list
        #self.ids[index]是根据index得到化学分子文件名
        #self.root是路径
        g, target = xyz_graph_reader(self.idx[index])
        if self.vertex_transform is not None:
            h = self.vertex_transform(g)

        if self.edge_transform is not None:
            g, e = self.edge_transform(g)

        if self.target_transform is not None:
            target = self.target_transform(target)

        #g：邻接矩阵
        #h：每个点的特征（list of list）
        #e：词典，key是边，value是特征
        #target：标签list
        return (g, h, e), target

    def __len__(self):
        return len(self.idx)

    def set_target_transform(self, target_transform):
        self.target_transform = target_transform

In [21]:
data_train = Qm9(train_ids, edge_transform=qm9_edges, e_representation='raw_distance')
data_valid = Qm9(valid_ids, edge_transform=qm9_edges, e_representation='raw_distance')
data_test = Qm9(test_ids, edge_transform=qm9_edges, e_representation='raw_distance')

In [22]:
# Define model and optimizer

In [23]:
# Select one graph
g_tuple, l = data_train[0]
g, h_t, e = g_tuple

In [24]:
def get_values(obj, start, end):
    vals = []
    for i in range(start, end):
        vals.append(obj[i][1])
    return vals
def get_graph_stats(graph_obj_handle):
    inputs = len(graph_obj_handle)
    res = get_values(graph_obj_handle, 0, inputs) 
    param = np.array(res)
    
    stat_dict = {}
    
    stat_dict['target_mean'] = np.mean(param, axis=0)
    stat_dict['target_std'] = np.std(param, axis=0)

    return stat_dict

In [25]:
stat_dict = get_graph_stats(data_valid)

In [26]:
# set_target_transform was defined in Class Qm9
data_train.set_target_transform(lambda x:x)
data_valid.set_target_transform(lambda x: x)
data_test.set_target_transform(lambda x: x)

# def normalize_data(data, mean, std):
#     data_norm = (data-mean)/std
#     return data_norm

#data_train.set_target_transform(lambda x: normalize_data(x,stat_dict['target_mean'],stat_dict['target_std']))
#data_valid.set_target_transform(lambda x: normalize_data(x, stat_dict['target_mean'],stat_dict['target_std']))
#data_test.set_target_transform(lambda x: normalize_data(x, stat_dict['target_mean'],stat_dict['target_std']))

In [27]:
# Data Loader

In [28]:
def collate_g(batch):

    batch_sizes = np.max(np.array([[len(input_b[1]), len(input_b[1][0]), len(input_b[2]),
                                len(list(input_b[2].values())[0])]
                                if input_b[2] else
                                [len(input_b[1]), len(input_b[1][0]), 0,0]
                                for (input_b, target_b) in batch]), axis=0)

    g = np.zeros((len(batch), batch_sizes[0], batch_sizes[0]))
    h = np.zeros((len(batch), batch_sizes[0], batch_sizes[1]))
    e = np.zeros((len(batch), batch_sizes[0], batch_sizes[0], batch_sizes[3]))

    target = np.zeros((len(batch), len(batch[0][1])))

    for i in range(len(batch)):

        num_nodes = len(batch[i][0][1])

        # Adjacency matrix
        g[i, 0:num_nodes, 0:num_nodes] = batch[i][0][0]

        # Node features
        h[i, 0:num_nodes, :] = batch[i][0][1]

        # Edges
        for edge in batch[i][0][2].keys():
            e[i, edge[0], edge[1], :] = batch[i][0][2][edge]
            e[i, edge[1], edge[0], :] = batch[i][0][2][edge]

        # Target
        target[i, :] = batch[i][1]

    g = torch.FloatTensor(g)
    h = torch.FloatTensor(h)
    e = torch.FloatTensor(e)
    target = torch.FloatTensor(target)

    return g, h, e, target

In [29]:

#注意这里面的collate_g函数,在utils里面
train_loader = torch.utils.data.DataLoader(data_train,
                                           batch_size=args.batch_size,shuffle=True, collate_fn=collate_g,
                                           num_workers=args.prefetch, pin_memory=True)
valid_loader = torch.utils.data.DataLoader(data_valid,
                                           batch_size=args.batch_size, collate_fn=collate_g,
                                           num_workers=args.prefetch, pin_memory=True)
test_loader = torch.utils.data.DataLoader(data_test,
                                          batch_size=args.batch_size, collate_fn=collate_g,
                                          num_workers=args.prefetch, pin_memory=True)

In [30]:
#define model

In [31]:
class NNet(nn.Module):

    def __init__(self, n_in, n_out, hlayers=(91, 256, 128)):
        super(NNet, self).__init__()
        self.n_hlayers = len(hlayers)
        self.fcs = nn.ModuleList([nn.Linear(n_in, hlayers[i]) if i == 0 else
                                  nn.Linear(hlayers[i-1], n_out) if i == self.n_hlayers else
                                  nn.Linear(hlayers[i-1], hlayers[i]) for i in range(self.n_hlayers+1)])

    def forward(self, x):
        x = x.contiguous().view(-1, self.num_flat_features(x))
        for i in range(self.n_hlayers):
            x = F.relu(self.fcs[i](x))
        x = self.fcs[-1](x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [32]:
#dtype = torch.cuda.FloatTensor
dtype = torch.FloatTensor

In [33]:
class MessageFunction(nn.Module):

    # Constructor
    def __init__(self, message_def='mpnn', args={}):
        super(MessageFunction, self).__init__()
        self.m_definition = ''
        self.m_function = None
        self.args = {}
        self.__set_message(message_def, args)

    # Message from h_v to h_w through e_vw
    def forward(self, h_v, h_w, e_vw, args=None):
        return self.m_function(h_v, h_w, e_vw, args)

    # Set a message function
    def __set_message(self, message_def, args={}):
        self.m_definition = message_def.lower()

        self.m_function = {
                    'duvenaud':         self.m_duvenaud,
                    'intnet':             self.m_intnet,
                    'mpnn':             self.m_mpnn,
                }.get(self.m_definition, None)

        if self.m_function is None:
            print('WARNING!: Message Function has not been set correctly\n\tIncorrect definition ' + message_def)
            quit()

        init_parameters = {
            'duvenaud': self.init_duvenaud,            
            'intnet':     self.init_intnet,
            'mpnn':     self.init_mpnn
        }.get(self.m_definition, lambda x: (nn.ParameterList([]), nn.ModuleList([]), {}))

        self.learn_args, self.learn_modules, self.args = init_parameters(args)

        self.m_size = {
                'duvenaud':     self.out_duvenaud,            
                'intnet':         self.out_intnet,
                'mpnn':         self.out_mpnn
            }.get(self.m_definition, None)

    # Get the name of the used message function
    def get_definition(self):
        return self.m_definition

    # Get the message function arguments
    def get_args(self):
        return self.args

    # Get Output size
    def get_out_size(self, size_h, size_e, args=None):
        return self.m_size(size_h, size_e, args)
    
    
    # Duvenaud et al. (2015), Convolutional Networks for Learning Molecular Fingerprints
    def m_duvenaud(self, h_v, h_w, e_vw, args):
        m = torch.cat([h_w, e_vw], 2)
        return m

    def out_duvenaud(self, size_h, size_e, args):
        return size_h + size_e

    def init_duvenaud(self, params):
        learn_args = []
        learn_modules = []
        args = {}
        return nn.ParameterList(learn_args), nn.ModuleList(learn_modules), args 

    # Battaglia et al. (2016), Interaction Networks
    def m_intnet(self, h_v, h_w, e_vw, args):
        m = torch.cat([h_v[:, None, :].expand_as(h_w), h_w, e_vw], 2)
        b_size = m.size()

        m = m.view(-1, b_size[2])

        m = self.learn_modules[0](m)
        m = m.view(b_size[0], b_size[1], -1)
        return m

    def out_intnet(self, size_h, size_e, args):
        return self.args['out']

    def init_intnet(self, params):
        learn_args = []
        learn_modules = []
        args = {}
        args['in'] = params['in']
        args['out'] = params['out']
        learn_modules.append(NNet(n_in=params['in'], n_out=params['out']))
        return nn.ParameterList(learn_args), nn.ModuleList(learn_modules), args

    # Gilmer et al. (2017), Neural Message Passing for Quantum Chemistry
    def m_mpnn(self, h_v, h_w, e_vw, opt={}):
        # Matrices for each edge
        edge_output = self.learn_modules[0](e_vw)
        edge_output = edge_output.view(-1, self.args['out'], self.args['in'])

        #h_w_rows = h_w[..., None].expand(h_w.size(0), h_v.size(1), h_w.size(1)).contiguous()
        h_w_rows = h_w[..., None].expand(h_w.size(0), h_w.size(1), h_v.size(1)).contiguous()

        h_w_rows = h_w_rows.view(-1, self.args['in'])

        h_multiply = torch.bmm(edge_output, torch.unsqueeze(h_w_rows,2))

        m_new = torch.squeeze(h_multiply)

        return m_new

    def out_mpnn(self, size_h, size_e, args):
        return self.args['out']

    def init_mpnn(self, params):
        learn_args = []
        learn_modules = []
        args = {}

        args['in'] = params['in']
        args['out'] = params['out']

        # Define a parameter matrix A for each edge label.
        learn_modules.append(NNet(n_in=params['edge_feat'], n_out=(params['in']*params['out'])))

        return nn.ParameterList(learn_args), nn.ModuleList(learn_modules), args

In [34]:
# UpdateFunction.py: Updates the nodes using the previous state and the message.

class UpdateFunction(nn.Module):

    # Constructor
    def __init__(self, update_def='nn', args={}):
        super(UpdateFunction, self).__init__()
        self.u_definition = ''
        self.u_function = None
        self.args = {}
        self.__set_update(update_def, args)

    # Update node hv given message mv
    def forward(self, h_v, m_v, opt={}):
        return self.u_function(h_v, m_v, opt)

    # Set update function
    def __set_update(self, update_def, args):
        self.u_definition = update_def.lower()

        self.u_function = {
                    'duvenaud':         self.u_duvenaud,            
                    'intnet':             self.u_intnet,
                    'mpnn':             self.u_mpnn
                }.get(self.u_definition, None)

        if self.u_function is None:
            print('WARNING!: Update Function has not been set correctly\n\tIncorrect definition ' + update_def)

        init_parameters = {
            'duvenaud':         self.init_duvenaud,            
            'intnet':             self.init_intnet,
            'mpnn':             self.init_mpnn
        }.get(self.u_definition, lambda x: (nn.ParameterList([]), nn.ModuleList([]), {}))

        self.learn_args, self.learn_modules, self.args = init_parameters(args)

    # Get the name of the used update function
    def get_definition(self):
        return self.u_definition

    # Get the update function arguments
    def get_args(self):
        return self.args
    
    # Duvenaud
    def u_duvenaud(self, h_v, m_v, opt):

        param_sz = self.learn_args[0][opt['deg']].size()
        parameter_mat = torch.t(self.learn_args[0][opt['deg']])[None, ...].expand(m_v.size(0), param_sz[1], param_sz[0])
        
        #print(parameter_mat.size())
        #print(m_v.size())
        #print(torch.transpose(m_v.unsqueeze(-2), 1, 2).size())

        #aux = torch.bmm(parameter_mat, torch.transpose(m_v, 1, 2))
        aux = torch.bmm(parameter_mat, torch.transpose(m_v.unsqueeze(-2), 1, 2))

        return torch.transpose(torch.nn.Sigmoid()(aux), 1, 2)

    def init_duvenaud(self, params):
        learn_args = []
        learn_modules = []
        args = {}

        # Filter degree 0 (the message will be 0 and therefore there is no update
        args['deg'] = [i for i in params['deg'] if i!=0]
        args['in'] = params['in']
        args['out'] = params['out']

        # Define a parameter matrix H for each degree.
        learn_args.append(torch.nn.Parameter(torch.randn(len(args['deg']), args['in'], args['out'])))

        return nn.ParameterList(learn_args), nn.ModuleList(learn_modules), args    

    # Battaglia et al. (2016), Interaction Networks
    def u_intnet(self, h_v, m_v, opt):
        if opt['x_v'].ndimension():
            input_tensor = torch.cat([h_v, opt['x_v'], torch.squeeze(m_v)], 1)
        else:
            input_tensor = torch.cat([h_v, torch.squeeze(m_v)], 1)

        return self.learn_modules[0](input_tensor)

    def init_intnet(self, params):
        learn_args = []
        learn_modules = []
        args = {}

        args['in'] = params['in']
        args['out'] = params['out']

        learn_modules.append(NNet(n_in=params['in'], n_out=params['out']))

        return nn.ParameterList(learn_args), nn.ModuleList(learn_modules), args

    def u_mpnn(self, h_v, m_v, opt={}):
        h_in = h_v.view(-1,h_v.size(2))
        m_in = m_v.view(-1,m_v.size(2))
        h_new = self.learn_modules[0](m_in[None,...],h_in[None,...])[0] # 0 or 1???
        return torch.squeeze(h_new).view(h_v.size())

    def init_mpnn(self, params):
        learn_args = []
        learn_modules = []
        args = {}

        args['in_m'] = params['in_m']
        args['out'] = params['out']

        # GRU
        learn_modules.append(nn.GRU(params['in_m'], params['out']))

        return nn.ParameterList(learn_args), nn.ModuleList(learn_modules), args

In [35]:
#ReadoutFunction
#dtype = torch.cuda.FloatTensor
dtype = torch.FloatTensor

class ReadoutFunction(nn.Module):

    # Constructor
    def __init__(self, readout_def='nn', args={}):
        super(ReadoutFunction, self).__init__()
        self.r_definition = ''
        self.r_function = None
        self.args = {}
        self.__set_readout(readout_def, args)

    # Readout graph given node values at las layer
    def forward(self, h_v):
        return self.r_function(h_v)

    # Set a readout function
    def __set_readout(self, readout_def, args):
        self.r_definition = readout_def.lower()

        self.r_function = {
                    'duvenaud': self.r_duvenaud,            
                    'intnet':     self.r_intnet,
                    'mpnn':     self.r_mpnn
                }.get(self.r_definition, None)

        if self.r_function is None:
            print('WARNING!: Readout Function has not been set correctly\n\tIncorrect definition ' + readout_def)
            quit()

        init_parameters = {
            'duvenaud': self.init_duvenaud,            
            'intnet':     self.init_intnet,
            'mpnn':     self.init_mpnn
        }.get(self.r_definition, lambda x: (nn.ParameterList([]), nn.ModuleList([]), {}))

        self.learn_args, self.learn_modules, self.args = init_parameters(args)

    # Get the name of the used readout function
    def get_definition(self):
        return self.r_definition

    # Duvenaud
    def r_duvenaud(self, h):
        # layers
        aux = []
        for l in range(len(h)):
            param_sz = self.learn_args[l].size()
            parameter_mat = torch.t(self.learn_args[l])[None, ...].expand(h[l].size(0), param_sz[1],
                                                                                      param_sz[0])

            aux.append(torch.transpose(torch.bmm(parameter_mat, torch.transpose(h[l], 1, 2)), 1, 2))

            for j in range(0, aux[l].size(1)):
                # Mask whole 0 vectors
                aux[l][:, j, :] = nn.Softmax()(aux[l][:, j, :].clone())*(torch.sum(aux[l][:, j, :] != 0, 1) > 0)[...,None].expand_as(aux[l][:, j, :]).type_as(aux[l])

        aux = torch.sum(torch.sum(torch.stack(aux, 3), 3), 1)
        return self.learn_modules[0](torch.squeeze(aux))

    def init_duvenaud(self, params):
        learn_args = []
        learn_modules = []
        args = {}

        args['out'] = params['out']

        # Define a parameter matrix W for each layer.
        for l in range(params['layers']):
            learn_args.append(nn.Parameter(torch.randn(params['in'][l], params['out'])))

        # learn_modules.append(nn.Linear(params['out'], params['target']))

        learn_modules.append(NNet(n_in=params['out'], n_out=params['target']))
        return nn.ParameterList(learn_args), nn.ModuleList(learn_modules), args    
    
    
    # Battaglia et al. (2016), Interaction Networks
    def r_intnet(self, h):

        aux = torch.sum(h[-1],1)

        return self.learn_modules[0](aux)

    def init_intnet(self, params):
        learn_args = []
        learn_modules = []
        args = {}

        learn_modules.append(NNet(n_in=params['in'], n_out=params['target']))

        return nn.ParameterList(learn_args), nn.ModuleList(learn_modules), args

    def r_mpnn(self, h):

        aux = Variable( torch.Tensor(h[0].size(0), self.args['out']).type_as(h[0].data).zero_() )
        # For each graph
        for i in range(h[0].size(0)):
            nn_res = nn.Sigmoid()(self.learn_modules[0](torch.cat([h[0][i,:,:], h[-1][i,:,:]], 1)))*self.learn_modules[1](h[-1][i,:,:])

            # Delete virtual nodes
            nn_res = (torch.sum(h[0][i,:,:],1)[...,None].expand_as(nn_res)>0).type_as(nn_res)* nn_res

            aux[i,:] = torch.sum(nn_res,0)

        return aux

    def init_mpnn(self, params):
        learn_args = []
        learn_modules = []
        args = {}

        # i
        learn_modules.append(NNet(n_in=2*params['in'], n_out=params['target']))

        # j
        learn_modules.append(NNet(n_in=params['in'], n_out=params['target']))

        args['out'] = params['target']

        return nn.ParameterList(learn_args), nn.ModuleList(learn_modules), args


In [36]:
#!/usr/bin/python

class MPNN(nn.Module):
    """
        MPNN as proposed by Gilmer et al..

        This class implements the whole Gilmer et al. model following the functions Message, Update and Readout.

        Parameters
        ----------
        in_n : int list
            Sizes for the node and edge features.
        hidden_state_size : int
            Size of the hidden states (the input will be padded with 0's to this size).
        message_size : int
            Message function output vector size.
        n_layers : int
            Number of iterations Message+Update (weight tying).
        l_target : int
            Size of the output.
        type : str (Optional)
            Classification | [Regression (default)]. If classification, LogSoftmax layer is applied to the output vector.
    """

    def __init__(self, in_n, hidden_state_size, message_size, n_layers, l_target, type='regression'):
        super(MPNN, self).__init__()

        # Define message
        self.m = nn.ModuleList(
            [MessageFunction('mpnn', args={'edge_feat': in_n[1], 'in': hidden_state_size, 
                                           'out': message_size})])

        # Define Update
        self.u = nn.ModuleList([UpdateFunction('mpnn',args={'in_m': message_size,
                                                            'out': hidden_state_size})])

        # Define Readout
        self.r = ReadoutFunction('mpnn',args={'in': hidden_state_size,
                                              'target': l_target})

        self.type = type

        self.args = {}
        self.args['out'] = hidden_state_size

        self.n_layers = n_layers

    def forward(self, g, h_in, e):

        h = []

        # Padding to some larger dimension d
        h_t = torch.cat([h_in, Variable(
            torch.zeros(h_in.size(0), h_in.size(1), self.args['out'] - h_in.size(2)).type_as(h_in.data))], 2)

        h.append(h_t.clone())

        # Layer
        for t in range(0, self.n_layers):
            e_aux = e.view(-1, e.size(3))

            h_aux = h[t].view(-1, h[t].size(2))

            m = self.m[0].forward(h[t], h_aux, e_aux)
            m = m.view(h[0].size(0), h[0].size(1), -1, m.size(1))

            # Nodes without edge set message to 0
            m = torch.unsqueeze(g, 3).expand_as(m) * m

            m = torch.squeeze(torch.sum(m, 1))

            h_t = self.u[0].forward(h[t], m)

            # Delete virtual nodes
            h_t = (torch.sum(h_in, 2)[..., None].expand_as(h_t) > 0).type_as(h_t) * h_t
            h.append(h_t)

        # Readout
        res = self.r.forward(h)
        if self.type == 'classification':
            res = F.softmax(res, dim=1)
        
        return res
    
#res1 = F.softmax(output, dim=1)
#res2 = nn.LogSoftmax()(output)

In [37]:
#Create model

In [38]:
from tensorboard_logger import configure, log_value
from torchmetrics import Recall

# def error_ratio(pred, target):
#     if type(pred) is not np.ndarray:
#         pred = np.array(pred)
#     if type(target) is not np.ndarray:
#         target = np.array(target)       
        
#     return np.mean(np.divide(np.abs(pred - target), np.abs(target)))

def pred(output):
    pred = F.softmax(output, dim=1)
    pred  = torch.where(pred> 0.5, 1, pred)
    pred  = torch.where(pred< 0.5, 0, pred)
    return pred



class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


class Logger(object):
    def __init__(self, log_dir):
        if not os.path.isdir(log_dir):
            # if the directory does not exist we create the directory
            os.makedirs(log_dir)
        else:                      
            # clean previous logged data under the same directory name
            self._remove(log_dir)

        # configure the project
        configure(log_dir)

        self.global_step = 0

    def log_value(self, name, value):
        log_value(name, value, self.global_step)
        return self

    def step(self):
        self.global_step += 1

    @staticmethod
    def _remove(path):
        """ param <path> could either be relative or absolute. """
        if os.path.isfile(path):
            os.remove(path)  # remove the file
        elif os.path.isdir(path):
            import shutil
            shutil.rmtree(path)  # remove dir and all contains

2022-11-17 04:51:16.169204: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-17 04:51:16.330954: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-17 04:51:16.365520: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-17 04:51:17.024383: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [39]:
from sklearn.utils.class_weight import compute_class_weight
lst_imbalance_ratio = []
for i in dat_label.values:
    class_weights=compute_class_weight('balanced',classes=np.unique(i), y=i)
    #print(np.bincount(i))
    #print(class_weights)
    #ratio = class_weights[1]/class_weights[0]
    ratio = class_weights[1]
    lst_imbalance_ratio.append(ratio)

In [40]:
import math
weights = []
for i in range(len(lst_imbalance_ratio)):
    weights.append(math.log(1+lst_imbalance_ratio[i]))
weights = torch.tensor(weights)

In [41]:


#点的特征维度，边的特征维度
in_n = [len(h_t[0]), len(list(e.values())[0])] 
#hidden state/embedding维度
hidden_state_size = 20
#邻居消息m_i维度（聚合后的维度）后面都用d_v表示
message_size = 20
#GNN层数
n_layers = 3
#labels数量
l_target = len(l)
#回归任务
type ='classification'
#type ='regression'

#定义mpnn模型
model = MPNN(in_n, hidden_state_size, message_size, n_layers, l_target, type=type)

del in_n, hidden_state_size, message_size, n_layers, l_target, type

#print('Optimizer')
optimizer = torch.optim.AdamW(model.parameters(), lr=args.lr)


#回归任务使用MSE 1ose
criterion = torch.nn.CrossEntropyLoss(weight=weights,reduction='mean')

#评估指标，|a-b|/|b|XXXXXXXXXXXXXXXXXXXX
#evaluation = lambda output, target: torch.mean(torch.abs(output - target) / torch.abs(target))

evaluation = lambda output, target: torch.eq(pred(output),target).float().mean()
#evaluation = lambda output, target: torch.eq(pred(output),target.squeeze(dim=-1)).float().mean()


#print('Logger')
logger = Logger(args.logPath)

lr_step = (args.lr-args.lr*args.lr_decay)/(args.epochs*args.schedule[1] - args.epochs*args.schedule[0])

#torch.backends.cudnn.enabled=False

In [42]:
def train(train_loader, model, criterion, optimizer, epoch, logger=None):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    accuracy = AverageMeter()

    # switch to train mode
    model.train()

    end = time.time()
    for i, (g, h, e, target) in enumerate(train_loader):

        # Prepare input data
        if args.cuda:
            g, h, e, target = g.cuda(), h.cuda(), e.cuda(), target.cuda()
        g, h, e, target = Variable(g), Variable(h), Variable(e), Variable(target)

        # Measure data loading time
        data_time.update(time.time() - end)

        optimizer.zero_grad()

        # Compute output
        output = model(g, h, e)
        train_loss = criterion(output, target)

        # Logs
        losses.update(train_loss.item(), g.size(0))
        accuracy.update(evaluation(output, target).item(), g.size(0))
        
        # compute gradient and do SGD step
        train_loss.backward()
        optimizer.step()

        # Measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % args.log_interval == 0 and i > 0:

            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Accuracy {err.val:.4f} ({err.avg:.4f})'
                  .format(epoch, i, len(train_loader), batch_time=batch_time,
                          data_time=data_time, loss=losses, err=accuracy))
                          
    logger.log_value('train_epoch_loss', losses.avg)
    logger.log_value('train_epoch_accuracy', accuracy.avg)

    print('Epoch: [{0}] Accuracy {err.avg:.3f}; Average Loss {loss.avg:.3f}; Avg Time x Batch {b_time.avg:.3f}'
          .format(epoch, err=accuracy, loss=losses, b_time=batch_time))

In [43]:
def validate(val_loader, model, criterion, evaluation, logger=None):
    batch_time = AverageMeter()
    losses = AverageMeter()
    accuracy = AverageMeter()
    #recall = AverageMeter()


    # switch to evaluate mode
    model.eval()

    end = time.time()
    for i, (g, h, e, target) in enumerate(val_loader):

        # Prepare input data
        if args.cuda:
            g, h, e, target = g.cuda(), h.cuda(), e.cuda(), target.cuda()
        g, h, e, target = Variable(g), Variable(h), Variable(e), Variable(target)

        # Compute output
        output = model(g, h, e)

        # Logs
        losses.update(criterion(output, target).item(), g.size(0))
        accuracy.update(evaluation(output, target).item(), g.size(0))
        #recall.update(evaluation_r(output, target).item(), g.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

#         if i % args.log_interval == 0 and i > 0:
            
#             print('Test: [{0}/{1}]\t'
#                   'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
#                   'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
#                   'Error Ratio {err.val:.4f} ({err.avg:.4f})'
#                   .format(i, len(val_loader), batch_time=batch_time,
#                           loss=losses, err=error_ratio))

    print(' * Average accuracy {err.avg:.3f}; '
          .format(err=accuracy))

    if logger is not None:
        logger.log_value('test_epoch_loss', losses.avg)
        logger.log_value('test_accuracy', accuracy.avg)
        #logger.log_value('test_recall', recall.avg)

    return accuracy.avg
    #return accuracy.avg,recall.avg

In [44]:
def save_checkpoint(state, is_best, directory):

    if not os.path.isdir(directory):
        os.makedirs(directory)
    checkpoint_file = os.path.join(directory, 'checkpoint.pth')
    best_model_file = os.path.join(directory, 'model_best.pth')
    torch.save(state, checkpoint_file)
    if is_best:
        shutil.copyfile(checkpoint_file, best_model_file)

In [45]:

# get the best checkpoint if available without training
if args.resume:
    checkpoint_dir = args.resume
    best_model_file = os.path.join(checkpoint_dir, 'model_best.pth')
    if not os.path.isdir(checkpoint_dir):
        os.makedirs(checkpoint_dir)
    if os.path.isfile(best_model_file):
        print("=> loading best model '{}'".format(best_model_file))
        checkpoint = torch.load(best_model_file)
        args.start_epoch = checkpoint['epoch']
        best_acc1 = checkpoint['best_er1']
        model.load_state_dict(checkpoint['state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        print("=> loaded best model '{}' (epoch {})".format(best_model_file, checkpoint['epoch']))
    else:
        print("=> no best model found at '{}'".format(best_model_file))

print('Check cuda')
if args.cuda:
    print('\t* Cuda')
    model = model.cuda()
    criterion = criterion.cuda()
    optimizer.load_state_dict(checkpoint['optimizer'])
    # 因为optimizer加载参数时,tensor默认在CPU上
    # 故需将所有的tensor都放到cuda,
    # 否则: 在optimizer.step()处报错：
    # RuntimeError: expected device cpu but got device cuda:0
    for state in optimizer.state.values():
        for k, v in state.items():
            if torch.is_tensor(v):
                state[k] = v.cuda()



=> loading best model './checkpoint/qm9/mpnn/model_best.pth'
=> loaded best model './checkpoint/qm9/mpnn/model_best.pth' (epoch 21)
Check cuda
	* Cuda


In [46]:
# Epoch for loop
for epoch in range(0, args.epochs):
    torch.cuda.empty_cache()

    if epoch > args.epochs * args.schedule[0] and epoch < args.epochs * args.schedule[1]:
        args.lr -= lr_step
        for param_group in optimizer.param_groups:
            param_group['lr'] = args.lr

    # train for one epoch
    train(train_loader, model, criterion, optimizer, epoch, logger)
    # evaluate on test set
    er1 = validate(valid_loader, model, criterion, evaluation,logger)

    is_best = er1 > best_er1
    best_er1 = min(er1, best_er1)
    save_checkpoint({'epoch': epoch + 1, 'state_dict': model.state_dict(), 'best_er1': best_er1,
                          'optimizer': optimizer.state_dict(), }, is_best=is_best, directory=args.resume)

    # Logger step
    logger.log_value('learning_rate', args.lr).step()

# get the best checkpoint and test it with test set
if args.resume:
    checkpoint_dir = args.resume
    best_model_file = os.path.join(checkpoint_dir, 'model_best.pth')
    if not os.path.isdir(checkpoint_dir):
        os.makedirs(checkpoint_dir)
    if os.path.isfile(best_model_file):
        print("=> loading best model '{}'".format(best_model_file))
        checkpoint = torch.load(best_model_file)
        args.start_epoch = checkpoint['epoch']
        best_acc1 = checkpoint['best_er1']
        model.load_state_dict(checkpoint['state_dict'])
        if args.cuda:
            model.cuda()
        optimizer.load_state_dict(checkpoint['optimizer'])
        print("=> loaded best model '{}' (epoch {})".format(best_model_file, checkpoint['epoch']))
    else:
        print("=> no best model found at '{}'".format(best_model_file))


Epoch: [0][20/448]	Time 0.038 (0.074)	Data 0.000 (0.010)	Loss 0.3860 (0.3383)	Accuracy 0.9000 (0.9190)
Epoch: [0][40/448]	Time 0.038 (0.060)	Data 0.000 (0.005)	Loss 0.3860 (0.3311)	Accuracy 0.9000 (0.9220)
Epoch: [0][60/448]	Time 0.041 (0.057)	Data 0.000 (0.004)	Loss 0.6364 (0.3327)	Accuracy 0.8000 (0.9213)
Epoch: [0][80/448]	Time 0.051 (0.057)	Data 0.000 (0.003)	Loss 0.1357 (0.3304)	Accuracy 1.0000 (0.9222)
Epoch: [0][100/448]	Time 0.046 (0.061)	Data 0.000 (0.002)	Loss 0.3860 (0.3315)	Accuracy 0.9000 (0.9218)
Epoch: [0][120/448]	Time 0.043 (0.059)	Data 0.000 (0.002)	Loss 0.3860 (0.3219)	Accuracy 0.9000 (0.9256)
Epoch: [0][140/448]	Time 0.042 (0.062)	Data 0.000 (0.002)	Loss 0.1357 (0.3239)	Accuracy 1.0000 (0.9248)
Epoch: [0][160/448]	Time 0.047 (0.059)	Data 0.000 (0.002)	Loss 0.6364 (0.3378)	Accuracy 0.8000 (0.9193)
Epoch: [0][180/448]	Time 0.053 (0.058)	Data 0.000 (0.001)	Loss 0.1357 (0.3418)	Accuracy 1.0000 (0.9177)
Epoch: [0][200/448]	Time 0.049 (0.059)	Data 0.000 (0.001)	Loss 0.135

Epoch: [3][240/448]	Time 0.077 (0.057)	Data 0.000 (0.001)	Loss 0.2181 (0.3181)	Accuracy 1.0000 (0.9253)
Epoch: [3][260/448]	Time 0.293 (0.057)	Data 0.001 (0.001)	Loss 0.4775 (0.3140)	Accuracy 0.8000 (0.9238)
Epoch: [3][280/448]	Time 0.041 (0.058)	Data 0.000 (0.001)	Loss 0.4406 (0.3104)	Accuracy 0.9000 (0.9231)
Epoch: [3][300/448]	Time 0.197 (0.058)	Data 0.001 (0.001)	Loss 0.1958 (0.3079)	Accuracy 0.9000 (0.9229)
Epoch: [3][320/448]	Time 0.077 (0.058)	Data 0.000 (0.001)	Loss 0.3692 (0.3046)	Accuracy 0.9000 (0.9231)
Epoch: [3][340/448]	Time 0.034 (0.058)	Data 0.000 (0.001)	Loss 0.2196 (0.3021)	Accuracy 0.8000 (0.9199)
Epoch: [3][360/448]	Time 0.054 (0.058)	Data 0.000 (0.001)	Loss 0.4214 (0.2998)	Accuracy 0.8000 (0.9177)
Epoch: [3][380/448]	Time 0.050 (0.058)	Data 0.000 (0.001)	Loss 0.4915 (0.2988)	Accuracy 0.9000 (0.9181)
Epoch: [3][400/448]	Time 0.044 (0.057)	Data 0.000 (0.001)	Loss 0.6149 (0.2960)	Accuracy 0.8000 (0.9182)
Epoch: [3][420/448]	Time 0.060 (0.056)	Data 0.000 (0.001)	Loss 0

Epoch: [6] Accuracy 0.934; Average Loss 0.234; Avg Time x Batch 0.055
 * Average accuracy 0.918; 
Epoch: [7][20/448]	Time 0.037 (0.068)	Data 0.000 (0.009)	Loss 0.3916 (0.2459)	Accuracy 0.9000 (0.9286)
Epoch: [7][40/448]	Time 0.052 (0.058)	Data 0.000 (0.005)	Loss 0.2188 (0.2306)	Accuracy 1.0000 (0.9293)
Epoch: [7][60/448]	Time 0.042 (0.053)	Data 0.000 (0.003)	Loss 0.2323 (0.2169)	Accuracy 0.9000 (0.9393)
Epoch: [7][80/448]	Time 0.036 (0.053)	Data 0.000 (0.003)	Loss 0.1384 (0.2162)	Accuracy 1.0000 (0.9383)
Epoch: [7][100/448]	Time 0.048 (0.060)	Data 0.000 (0.002)	Loss 0.2285 (0.2164)	Accuracy 0.9000 (0.9406)
Epoch: [7][120/448]	Time 0.045 (0.058)	Data 0.000 (0.002)	Loss 0.1435 (0.2183)	Accuracy 1.0000 (0.9380)
Epoch: [7][140/448]	Time 0.043 (0.057)	Data 0.000 (0.002)	Loss 0.1362 (0.2202)	Accuracy 1.0000 (0.9369)
Epoch: [7][160/448]	Time 0.038 (0.057)	Data 0.000 (0.001)	Loss 0.1362 (0.2211)	Accuracy 1.0000 (0.9398)
Epoch: [7][180/448]	Time 0.038 (0.058)	Data 0.000 (0.001)	Loss 0.2674 (0.2

Epoch: [10][220/448]	Time 0.063 (0.057)	Data 0.000 (0.001)	Loss 0.2821 (0.2279)	Accuracy 1.0000 (0.9412)
Epoch: [10][240/448]	Time 0.058 (0.056)	Data 0.000 (0.001)	Loss 0.1853 (0.2280)	Accuracy 1.0000 (0.9432)
Epoch: [10][260/448]	Time 0.035 (0.056)	Data 0.000 (0.001)	Loss 0.4354 (0.2268)	Accuracy 0.9000 (0.9448)
Epoch: [10][280/448]	Time 0.072 (0.057)	Data 0.000 (0.001)	Loss 0.6868 (0.2312)	Accuracy 0.8000 (0.9420)
Epoch: [10][300/448]	Time 0.055 (0.058)	Data 0.000 (0.001)	Loss 0.1747 (0.2319)	Accuracy 0.9000 (0.9372)
Epoch: [10][320/448]	Time 0.040 (0.057)	Data 0.000 (0.001)	Loss 0.1817 (0.2324)	Accuracy 0.9000 (0.9355)
Epoch: [10][340/448]	Time 0.113 (0.057)	Data 0.001 (0.001)	Loss 0.3338 (0.2312)	Accuracy 1.0000 (0.9372)
Epoch: [10][360/448]	Time 0.040 (0.057)	Data 0.000 (0.001)	Loss 0.1381 (0.2289)	Accuracy 1.0000 (0.9382)
Epoch: [10][380/448]	Time 0.045 (0.056)	Data 0.000 (0.001)	Loss 0.2357 (0.2288)	Accuracy 1.0000 (0.9394)
Epoch: [10][400/448]	Time 0.043 (0.056)	Data 0.000 (0.0

Epoch: [13][420/448]	Time 0.040 (0.056)	Data 0.000 (0.001)	Loss 0.4000 (0.2305)	Accuracy 0.9000 (0.9416)
Epoch: [13][440/448]	Time 0.029 (0.056)	Data 0.000 (0.001)	Loss 0.1908 (0.2295)	Accuracy 1.0000 (0.9420)
Epoch: [13] Accuracy 0.941; Average Loss 0.229; Avg Time x Batch 0.055
 * Average accuracy 0.920; 
Epoch: [14][20/448]	Time 0.037 (0.080)	Data 0.000 (0.011)	Loss 0.1847 (0.2365)	Accuracy 0.9000 (0.9381)
Epoch: [14][40/448]	Time 0.049 (0.064)	Data 0.000 (0.006)	Loss 0.2490 (0.2414)	Accuracy 0.8000 (0.9415)
Epoch: [14][60/448]	Time 0.044 (0.061)	Data 0.000 (0.004)	Loss 0.2284 (0.2224)	Accuracy 0.9000 (0.9393)
Epoch: [14][80/448]	Time 0.038 (0.060)	Data 0.000 (0.003)	Loss 0.3748 (0.2250)	Accuracy 0.9000 (0.9432)
Epoch: [14][100/448]	Time 0.063 (0.060)	Data 0.000 (0.003)	Loss 0.1851 (0.2295)	Accuracy 1.0000 (0.9465)
Epoch: [14][120/448]	Time 0.048 (0.059)	Data 0.000 (0.002)	Loss 0.2141 (0.2269)	Accuracy 0.8000 (0.9380)
Epoch: [14][140/448]	Time 0.052 (0.060)	Data 0.000 (0.002)	Loss 0

...

Epoch: [184][260/448]	Time 0.031 (0.059)	Data 0.000 (0.001)	Loss 0.2285 (0.2288)	Accuracy 0.9000 (0.9444)
Epoch: [184][280/448]	Time 0.035 (0.058)	Data 0.000 (0.001)	Loss 0.1774 (0.2305)	Accuracy 0.9000 (0.9445)
Epoch: [184][300/448]	Time 0.052 (0.058)	Data 0.000 (0.001)	Loss 0.1855 (0.2286)	Accuracy 1.0000 (0.9449)
Epoch: [184][320/448]	Time 0.036 (0.057)	Data 0.000 (0.001)	Loss 0.1852 (0.2284)	Accuracy 0.9000 (0.9411)
Epoch: [184][340/448]	Time 0.032 (0.058)	Data 0.000 (0.001)	Loss 0.2346 (0.2291)	Accuracy 1.0000 (0.9408)
Epoch: [184][360/448]	Time 0.045 (0.057)	Data 0.000 (0.001)	Loss 0.3839 (0.2280)	Accuracy 0.9000 (0.9404)
Epoch: [184][380/448]	Time 0.050 (0.056)	Data 0.000 (0.001)	Loss 0.1358 (0.2244)	Accuracy 1.0000 (0.9428)
Epoch: [184][400/448]	Time 0.059 (0.056)	Data 0.000 (0.001)	Loss 0.2294 (0.2243)	Accuracy 0.9000 (0.9434)
Epoch: [184][420/448]	Time 0.057 (0.056)	Data 0.000 (0.001)	Loss 0.2358 (0.2248)	Accuracy 1.0000 (0.9435)
Epoch: [184][440/448]	Time 0.038 (0.055)	Data 

Epoch: [187][440/448]	Time 0.033 (0.055)	Data 0.000 (0.001)	Loss 0.1436 (0.2196)	Accuracy 1.0000 (0.9463)
Epoch: [187] Accuracy 0.946; Average Loss 0.221; Avg Time x Batch 0.055
 * Average accuracy 0.936; 
Epoch: [188][20/448]	Time 0.038 (0.072)	Data 0.000 (0.009)	Loss 0.1508 (0.2296)	Accuracy 1.0000 (0.9571)
Epoch: [188][40/448]	Time 0.150 (0.063)	Data 0.001 (0.005)	Loss 0.2812 (0.2308)	Accuracy 0.9000 (0.9463)
Epoch: [188][60/448]	Time 0.038 (0.064)	Data 0.000 (0.003)	Loss 0.1358 (0.2218)	Accuracy 1.0000 (0.9475)
Epoch: [188][80/448]	Time 0.048 (0.063)	Data 0.000 (0.003)	Loss 0.1357 (0.2270)	Accuracy 1.0000 (0.9506)
Epoch: [188][100/448]	Time 0.056 (0.063)	Data 0.000 (0.002)	Loss 0.1792 (0.2262)	Accuracy 0.9000 (0.9495)
Epoch: [188][120/448]	Time 0.043 (0.061)	Data 0.000 (0.002)	Loss 0.1790 (0.2226)	Accuracy 0.9000 (0.9488)
Epoch: [188][140/448]	Time 0.039 (0.060)	Data 0.000 (0.002)	Loss 0.1883 (0.2246)	Accuracy 1.0000 (0.9475)
Epoch: [188][160/448]	Time 0.050 (0.058)	Data 0.000 (0.0

Epoch: [191][160/448]	Time 0.055 (0.053)	Data 0.000 (0.001)	Loss 0.2286 (0.2316)	Accuracy 0.9000 (0.9429)
Epoch: [191][180/448]	Time 0.037 (0.057)	Data 0.000 (0.001)	Loss 0.2866 (0.2351)	Accuracy 0.7000 (0.9409)
Epoch: [191][200/448]	Time 0.150 (0.057)	Data 0.001 (0.001)	Loss 0.1793 (0.2332)	Accuracy 0.9000 (0.9408)
Epoch: [191][220/448]	Time 0.045 (0.057)	Data 0.000 (0.001)	Loss 0.3860 (0.2341)	Accuracy 0.9000 (0.9407)
Epoch: [191][240/448]	Time 0.200 (0.057)	Data 0.001 (0.001)	Loss 0.1851 (0.2316)	Accuracy 1.0000 (0.9419)
Epoch: [191][260/448]	Time 0.052 (0.057)	Data 0.000 (0.001)	Loss 0.2375 (0.2338)	Accuracy 1.0000 (0.9433)
Epoch: [191][280/448]	Time 0.088 (0.057)	Data 0.000 (0.001)	Loss 0.4849 (0.2313)	Accuracy 0.9000 (0.9441)
Epoch: [191][300/448]	Time 0.037 (0.057)	Data 0.000 (0.001)	Loss 0.4350 (0.2298)	Accuracy 0.9000 (0.9439)
Epoch: [191][320/448]	Time 0.063 (0.057)	Data 0.000 (0.001)	Loss 0.2298 (0.2273)	Accuracy 0.9000 (0.9436)
Epoch: [191][340/448]	Time 0.114 (0.056)	Data 

Epoch: [194][340/448]	Time 0.044 (0.057)	Data 0.000 (0.001)	Loss 0.2283 (0.2133)	Accuracy 0.9000 (0.9575)
Epoch: [194][360/448]	Time 0.037 (0.056)	Data 0.000 (0.001)	Loss 0.2447 (0.2143)	Accuracy 1.0000 (0.9576)
Epoch: [194][380/448]	Time 0.038 (0.056)	Data 0.000 (0.001)	Loss 0.2355 (0.2144)	Accuracy 0.9000 (0.9554)
Epoch: [194][400/448]	Time 0.041 (0.056)	Data 0.000 (0.001)	Loss 0.4270 (0.2145)	Accuracy 0.8000 (0.9551)
Epoch: [194][420/448]	Time 0.045 (0.055)	Data 0.000 (0.001)	Loss 0.3183 (0.2168)	Accuracy 0.9000 (0.9537)
Epoch: [194][440/448]	Time 0.039 (0.055)	Data 0.000 (0.001)	Loss 0.2078 (0.2168)	Accuracy 1.0000 (0.9535)
Epoch: [194] Accuracy 0.953; Average Loss 0.218; Avg Time x Batch 0.055
 * Average accuracy 0.938; 
Epoch: [195][20/448]	Time 0.036 (0.059)	Data 0.000 (0.007)	Loss 0.1358 (0.2089)	Accuracy 1.0000 (0.9619)
Epoch: [195][40/448]	Time 0.040 (0.055)	Data 0.000 (0.004)	Loss 0.1824 (0.2409)	Accuracy 0.9000 (0.9463)
Epoch: [195][60/448]	Time 0.038 (0.053)	Data 0.000 (0.

Epoch: [198][60/448]	Time 0.031 (0.052)	Data 0.000 (0.003)	Loss 0.1359 (0.2175)	Accuracy 1.0000 (0.9639)
Epoch: [198][80/448]	Time 0.034 (0.057)	Data 0.000 (0.002)	Loss 0.1649 (0.2139)	Accuracy 0.9000 (0.9617)
Epoch: [198][100/448]	Time 0.040 (0.058)	Data 0.000 (0.002)	Loss 0.1357 (0.2300)	Accuracy 1.0000 (0.9535)
Epoch: [198][120/448]	Time 0.125 (0.057)	Data 0.001 (0.002)	Loss 0.2757 (0.2245)	Accuracy 0.9000 (0.9554)
Epoch: [198][140/448]	Time 0.041 (0.055)	Data 0.000 (0.002)	Loss 0.1357 (0.2222)	Accuracy 1.0000 (0.9560)
Epoch: [198][160/448]	Time 0.160 (0.058)	Data 0.001 (0.001)	Loss 0.2264 (0.2250)	Accuracy 0.9000 (0.9534)
Epoch: [198][180/448]	Time 0.042 (0.058)	Data 0.000 (0.001)	Loss 0.3862 (0.2239)	Accuracy 0.9000 (0.9541)
Epoch: [198][200/448]	Time 0.039 (0.058)	Data 0.000 (0.001)	Loss 0.1767 (0.2217)	Accuracy 0.9000 (0.9557)
Epoch: [198][220/448]	Time 0.047 (0.058)	Data 0.000 (0.001)	Loss 0.1367 (0.2217)	Accuracy 1.0000 (0.9561)
Epoch: [198][240/448]	Time 0.129 (0.057)	Data 0.

In [47]:
torch.cuda.empty_cache()

In [48]:
from sklearn.metrics import roc_auc_score
def auc_score(target,preds):
    for i in range(len(target)):
        auc_score = roc_auc_score(target[i].cpu().detach().numpy(), preds[i].cpu().detach().numpy())
        auc_score = auc_score * len(target)/len(data_test)
    return auc_score    
    auc = auc_score(target,preds)
    aucs = aucs + auc

In [ ]:
# For testing
report = []
all_pred = torch.empty(0,2).cuda()
all_target = torch.empty(0,2).cuda()
batch_time = AverageMeter()
data_time = AverageMeter()
losses = AverageMeter()
accuracy = AverageMeter()
#auc = AverageMeter()

if args.cuda:
    print('\t* Cuda')
    model = model.cuda()
    criterion = criterion.cuda()
for i, (g, h, e, target) in enumerate(test_loader):

    # Prepare input data
    if args.cuda:
        g, h, e, target = g.cuda(), h.cuda(), e.cuda(), target.cuda()
    g, h, e, target = Variable(g), Variable(h), Variable(e), Variable(target)

    # Compute output
    output = model(g, h, e)

    preds = pred(output) 
    

    all_pred = torch.cat((all_pred,preds),0)
    all_target = torch.cat((all_target,target),0)
    #cl = classification_report(target.cpu().detach().numpy(), preds.cpu().detach().numpy(),output_dict=True)
    #auc_score = roc_auc_score(target.cpu().detach().numpy(), preds.cpu().detach().numpy())
    #print(cl)
    #report.append(cl)
    
    # Logs
    losses.update(criterion(output, target).item(), g.size(0))
    accuracy.update(evaluation(output, target).item(), g.size(0))
    #auc.update(roc_auc_score(target.cpu().detach().numpy(), preds.cpu().detach().numpy()), g.size(0))
#         if i % args.log_interval == 0 and i > 0:
            
#             print('Test: [{0}/{1}]\t'
#                   'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
#                   'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
#                   'Error Ratio {err.val:.4f} ({err.avg:.4f})'
#                   .format(i, len(val_loader), batch_time=batch_time,
#                           loss=losses, err=error_ratio))

    #print(' * Average accuracy {err.avg:.3f};Average recall {rec.avg:.3f}; Average Loss {loss.avg:.3f}'
    #      .format(err=accuracy,rec = recall, loss=losses))

    if logger is not None:
        logger.log_value('test_epoch_loss', losses.avg)
        logger.log_value('test_accuracy', accuracy.avg)

In [50]:
len(all_target)

200

In [51]:
len(all_pred)

200

In [52]:
all_target.t()[0]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.], device='cuda:0')

In [53]:
all_pred.t()[0]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 1., 0., 1.,
        1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.], device='cuda:0', grad_fn=<SelectBackward0>)

In [54]:
cl = classification_report(all_target.t()[0].cpu().detach().numpy(), all_pred.t()[0].cpu().detach().numpy(),output_dict=True)

In [55]:
roc_auc_score(all_target.t()[0].cpu().detach().numpy(), all_pred.t()[0].cpu().detach().numpy())

0.893625192012289

In [66]:
from sklearn.metrics import precision_recall_curve

In [67]:
precision, recall, thresholds = precision_recall_curve(all_target.t()[0].cpu().detach().numpy(), all_pred.t()[0].cpu().detach().numpy())

In [72]:
precision

array([0.07, 0.48, 1.  ])

In [56]:
cl

{'0.0': {'precision': 0.9885714285714285,
  'recall': 0.9301075268817204,
  'f1-score': 0.9584487534626038,
  'support': 186},
 '1.0': {'precision': 0.48,
  'recall': 0.8571428571428571,
  'f1-score': 0.6153846153846153,
  'support': 14},
 'accuracy': 0.925,
 'macro avg': {'precision': 0.7342857142857142,
  'recall': 0.8936251920122887,
  'f1-score': 0.7869166844236095,
  'support': 200},
 'weighted avg': {'precision': 0.9529714285714286,
  'recall': 0.925,
  'f1-score': 0.9344342637971447,
  'support': 200}}

In [59]:
cols = []
for i in list(cl.keys()):
    cols.append('label' + i)
df_report = pd.DataFrame(index = cols[:91],columns=report[0]['0'].keys())
df_report = df_report.fillna(0)

In [61]:
for e in range(len(report)):
    for l in range(2):
        label = '{}'.format(l)
        support = report[e][label]['support']
        if support != 0:
            df_report.iloc[l,0] = df_report.iloc[l,0]+ support * report[e][label]['precision']
            #print(report[e][label]['precision'],report[e][label]['recall'],report[e][label]['f1-score'],report[e][label]['support'])
            df_report.iloc[l,1] = df_report.iloc[l,1] + support * report[e][label]['recall']
            df_report.iloc[l,2] = df_report.iloc[l,2]  + support * report[e][label]['f1-score']
            df_report.iloc[l,3] = df_report.iloc[l,3] + report[e][label]['support']

In [62]:
df_report.to_csv('report.csv')

In [63]:
df_report

,precision,recall,f1-score,support
label0,7.000,4,5.000000,8
label1,113.165,117,115.042119,117
labelmicro avg,0.000,0,0.000000,0
labelmacro avg,0.000,0,0.000000,0
labelweighted avg,0.000,0,0.000000,0
labelsamples avg,0.000,0,0.000000,0


In [40]:
print(next(model.parameters()).device) 
data = torch.ones([0])
print(data.device) 

cuda:0
cpu
